# STACKOVERFLOW con Scrapy

"""
OBJETIVO: 
    - Extraer las preguntas de la pagina principal de Stackoverflow con Scrapy
CREADO POR: LEONARDO KUFFO
ULTIMA VEZ EDITADO: 09 ENERO 2023
"""

In [1]:
# VER RECURSOS DE LA CLASE PARA INSTALAR SCRAPY
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import Spider
from scrapy.selector import Selector
from scrapy.loader.processors import MapCompose
from scrapy.loader import ItemLoader
from bs4 import BeautifulSoup

In [2]:
#from scrapy.spiders import Spider
from scrapy.crawler import CrawlerProcess

Vamos a definir las clases que se necesitan en Scrapy

In [3]:
# ABSTRACCION DE DATOS A EXTRAER - DETERMINA LOS DATOS QUE TENGO QUE LLENAR Y QUE ESTARAN EN EL ARCHIVO GENERADO
class Pregunta(Item):
    id = Field()
    pregunta = Field()
    #descripcion = Field()

In [4]:
# CLASE CORE - SPIDER
class StackOverflowSpider(Spider):
    name = "MiPrimerSpider" # nombre, puede ser cualquiera 
    
    # Forma de configurar el USER AGENT en scrapy
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 Safari/537.36'
    }
    
    # URL SEMILLA
    start_urls = ['https://stackoverflow.com/questions']
    
    # Funcion que se va a llamar cuando se haga el requerimiento a la URL semilla
    def parse(self, response):
        # Selectores: Clase de scrapy para extraer datos
        sel = Selector(response) #Va a utilizar selectores de xpath
        titulo_de_pagina = sel.xpath('//h1/text()').get()
        print (titulo_de_pagina)
        # Selector de varias preguntas
        preguntas = sel.xpath('//div[@id="questions"]//div[contains(@class,"s-post-summary ")]') 
        i = 0
        for pregunta in preguntas:
            item = ItemLoader(Pregunta(), pregunta) # Instancio mi ITEM con el selector en donde estan los datos para llenarlo

            # Lleno las propiedades de mi ITEM a traves de expresiones XPATH a buscar dentro del selector "pregunta"
            item.add_xpath('pregunta', './/h3/a/text()') 
            # item.add_xpath('descripcion', './/div[@class="s-post-summary--content-excerpt"]/text()')
            item.add_value('id', i)
            i += 1
            yield item.load_item() # Hago Yield de la informacion para que se escriban los datos en el archivo

Ahora el problema es que no se puede correr como un notebook. Hay que correrlo por Terminal:  
> scrapy runspider nombredelarchivo.py -o nombrenuevoarchivo.csv -t csv

La forma de hacerlo es por medio de la carga de librería **from scrapy.crawler import CrawlerProcess**, y las siguientes instrucciones.

In [5]:
process = CrawlerProcess({
    'FEED_FORMAT': 'csv',
    'FEED_URI':'nombrenuevoarchivoipynb.csv'
})
process.crawl(StackOverflowSpider)
process.start()

2023-08-23 19:40:19 [scrapy.utils.log] INFO: Scrapy 2.10.0 started (bot: scrapybot)
2023-08-23 19:40:19 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.11.5, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.4 | packaged by conda-forge | (main, Jun 10 2023, 18:10:28) [Clang 15.0.7 ], pyOpenSSL 23.2.0 (OpenSSL 3.1.2 1 Aug 2023), cryptography 41.0.3, Platform macOS-12.6.8-x86_64-i386-64bit
2023-08-23 19:40:19 [scrapy.addons] INFO: Enabled addons:
[]
2023-08-23 19:40:19 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, '
               'like Gecko) Ubuntu Chromium/71.0.3578.80 Chrome/71.0.3578.80 '
               'Safari/537.36'}
2023-08-23 19:40:19 [py.warnings] WARNING: /opt/anaconda3/envs/webchanges/lib/python3.11/site-packages/scrapy/utils/request.py:248: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the defa

All Questions


2023-08-23 19:40:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://stackoverflow.com/questions>
{'id': [12], 'pregunta': ['Keycloak client name issue']}
2023-08-23 19:40:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://stackoverflow.com/questions>
{'id': [13],
 'pregunta': ['Order of operations with expr1=expr2 where LHS is an '
              'unordered_map value [duplicate]']}
2023-08-23 19:40:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://stackoverflow.com/questions>
{'id': [14],
 'pregunta': ['windows update Updateorchestrator restart after installation of '
              'updates Server 2016']}
2023-08-23 19:40:21 [scrapy.core.engine] INFO: Closing spider (finished)
2023-08-23 19:40:21 [scrapy.extensions.feedexport] INFO: Stored csv feed (15 items) in: nombrenuevoarchivoipynb.csv
2023-08-23 19:40:21 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 329,
 'downloader/request_count': 1,
 'downloader/request_method_count/GET'